In [16]:
import pandas as pd
import numpy as np
import json
import re
import os
import fnmatch
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [77]:
""" 1. list folder chatx DONE
    2. list file dalam folder
    
    EXPECTED:
    current dir + /parsed doc/ + chatx/ + sesix.txt    
"""
folder = os.getcwd() + '/parsed doc/'
chat_folders = [chat_folder for chat_folder in os.listdir(folder) if not chat_folder.startswith('.')]

paths = []
for i in range(len(chat_folders)):
    pattern = str(folder) + chat_folders[i]
    paths.append(pattern)
    
sesi_paths = []
for i in range(len(paths)):
    for roots, dirs, files in os.walk(paths[i]):
        if "checkpoints" not in roots:
            for file in files:
                sesi_paths.append(roots + '/' + file)

478

In [119]:
chats = [pd.read_csv(sesi, sep='|', names = ['sender', 'messages', 'sentiment']) for sesi in sesi_paths]

messages = [chats[i].messages.values.tolist() for i in range(len(sesi_paths))]

messages[477]

['Selamat siang, Pak [DOSEN]. Saya  salah satu mahasiswi bimbingan skripsi bapak. Saat ini skripsi saya baru sampai tahap P1, saya masih dalam tahap pembuatan prototype, Pak. Saya berencana P2 di minggu ini. Namun apabila dokumen skripsi saya masih belum selesai, dapatkah saya melanjutkan skripsi ini di semester genap, Pak? Dengan bapak dan Bu [DOSEN] sebagai pembimbing saya seperti saat ini. Mohon bantuannya Pak. Terimakasih sebelumnya, mohon maaf mengganggu waktunya. Selamat siang.',
 'walaikumsallam, dikerjakan progres realnya saja, nanti masalah administrasi sistem bisa diatur. yang penting kerjaan skripsinya tidak jauh2 dari timeline target kemarin, jadi semisal semester depan lanjut silahkan saja',
 'Baik Pak [DOSEN], terimakasih. Akan segera saya selesaikan.']

In [79]:
# sender = chats['sender'].values.tolist()
# messages = chats['messages'].values.tolist()
# sentiment = chats['sentiment'].values.tolist()

TypeError: list indices must be integers or slices, not str

In [122]:
def tokenize(msg):
    token = [re.split('\s+', msg[i]) for i, chat in enumerate(msg)]
    return token

token = tokenize(messages[477])
token

[['Selamat',
  'siang,',
  'Pak',
  '[DOSEN].',
  'Saya',
  'salah',
  'satu',
  'mahasiswi',
  'bimbingan',
  'skripsi',
  'bapak.',
  'Saat',
  'ini',
  'skripsi',
  'saya',
  'baru',
  'sampai',
  'tahap',
  'P1,',
  'saya',
  'masih',
  'dalam',
  'tahap',
  'pembuatan',
  'prototype,',
  'Pak.',
  'Saya',
  'berencana',
  'P2',
  'di',
  'minggu',
  'ini.',
  'Namun',
  'apabila',
  'dokumen',
  'skripsi',
  'saya',
  'masih',
  'belum',
  'selesai,',
  'dapatkah',
  'saya',
  'melanjutkan',
  'skripsi',
  'ini',
  'di',
  'semester',
  'genap,',
  'Pak?',
  'Dengan',
  'bapak',
  'dan',
  'Bu',
  '[DOSEN]',
  'sebagai',
  'pembimbing',
  'saya',
  'seperti',
  'saat',
  'ini.',
  'Mohon',
  'bantuannya',
  'Pak.',
  'Terimakasih',
  'sebelumnya,',
  'mohon',
  'maaf',
  'mengganggu',
  'waktunya.',
  'Selamat',
  'siang.'],
 ['walaikumsallam,',
  'dikerjakan',
  'progres',
  'realnya',
  'saja,',
  'nanti',
  'masalah',
  'administrasi',
  'sistem',
  'bisa',
  'diatur.',
  'yang

In [123]:
def casefolding(msg):
    token_lower = [[token.lower() for token in i if token] for i in msg]
    return token_lower

token_lower = casefolding(token)
token_lower

[['selamat',
  'siang,',
  'pak',
  '[dosen].',
  'saya',
  'salah',
  'satu',
  'mahasiswi',
  'bimbingan',
  'skripsi',
  'bapak.',
  'saat',
  'ini',
  'skripsi',
  'saya',
  'baru',
  'sampai',
  'tahap',
  'p1,',
  'saya',
  'masih',
  'dalam',
  'tahap',
  'pembuatan',
  'prototype,',
  'pak.',
  'saya',
  'berencana',
  'p2',
  'di',
  'minggu',
  'ini.',
  'namun',
  'apabila',
  'dokumen',
  'skripsi',
  'saya',
  'masih',
  'belum',
  'selesai,',
  'dapatkah',
  'saya',
  'melanjutkan',
  'skripsi',
  'ini',
  'di',
  'semester',
  'genap,',
  'pak?',
  'dengan',
  'bapak',
  'dan',
  'bu',
  '[dosen]',
  'sebagai',
  'pembimbing',
  'saya',
  'seperti',
  'saat',
  'ini.',
  'mohon',
  'bantuannya',
  'pak.',
  'terimakasih',
  'sebelumnya,',
  'mohon',
  'maaf',
  'mengganggu',
  'waktunya.',
  'selamat',
  'siang.'],
 ['walaikumsallam,',
  'dikerjakan',
  'progres',
  'realnya',
  'saja,',
  'nanti',
  'masalah',
  'administrasi',
  'sistem',
  'bisa',
  'diatur.',
  'yang

In [124]:
def cleaning(msg):
    link = r'(www\.[^\s]+)|(https?:\/\/[^\s]+)|(meet\.google\.[^\s]+)'
    email = r'([a-zA-Z0-9\.\_\-]+@+[a-zA-Z0-9.]+)'
    punct = r'[^a-zA-Z0-9\-\s\[\]<>]'
    
    token_clean = []
    for i in msg:
        token_per_chat = []
        for token in i:
            # remove link
            temp = re.sub(link, '', token)
            # remove email
            temp = re.sub(email, '', temp)
            # remove punctuation
            temp = re.sub(punct, '', temp)
            temp = re.sub(r'\-', ' ', temp)
            # remove numbers
            temp = re.sub(r'\b[0-9]+\b\s*', '', temp)
            token_per_chat.append(temp)
        token_clean.append(token_per_chat)
        while('' in token_per_chat): 
            token_per_chat.remove('')
    return token_clean

token_clean = cleaning(token_lower)
token_clean

[['selamat',
  'siang',
  'pak',
  '[dosen]',
  'saya',
  'salah',
  'satu',
  'mahasiswi',
  'bimbingan',
  'skripsi',
  'bapak',
  'saat',
  'ini',
  'skripsi',
  'saya',
  'baru',
  'sampai',
  'tahap',
  'p1',
  'saya',
  'masih',
  'dalam',
  'tahap',
  'pembuatan',
  'prototype',
  'pak',
  'saya',
  'berencana',
  'p2',
  'di',
  'minggu',
  'ini',
  'namun',
  'apabila',
  'dokumen',
  'skripsi',
  'saya',
  'masih',
  'belum',
  'selesai',
  'dapatkah',
  'saya',
  'melanjutkan',
  'skripsi',
  'ini',
  'di',
  'semester',
  'genap',
  'pak',
  'dengan',
  'bapak',
  'dan',
  'bu',
  '[dosen]',
  'sebagai',
  'pembimbing',
  'saya',
  'seperti',
  'saat',
  'ini',
  'mohon',
  'bantuannya',
  'pak',
  'terimakasih',
  'sebelumnya',
  'mohon',
  'maaf',
  'mengganggu',
  'waktunya',
  'selamat',
  'siang'],
 ['walaikumsallam',
  'dikerjakan',
  'progres',
  'realnya',
  'saja',
  'nanti',
  'masalah',
  'administrasi',
  'sistem',
  'bisa',
  'diatur',
  'yang',
  'penting',
  

In [125]:
def normalize(msg):
    with open('slang_words.json', 'r') as f:
        dict = json.load(f)
        
    normalized_chat = []
    for i in msg:
        slang_dict = {v:k for v, k in dict.items()}
        token_per_chat = []
        for token in i:
            normal = slang_dict.get(token, token)
            token_per_chat.append(normal)
        normalized_chat.append(token_per_chat)
    return normalized_chat

token_normal = normalize(token_clean)
token_normal

[['selamat',
  'siang',
  'bapak',
  '[dosen]',
  'saya',
  'salah',
  'satu',
  'mahasiswi',
  'bimbingan',
  'skripsi',
  'bapak',
  'saat',
  'ini',
  'skripsi',
  'saya',
  'baru',
  'sampai',
  'tahap',
  'p1',
  'saya',
  'masih',
  'dalam',
  'tahap',
  'pembuatan',
  'prototype',
  'bapak',
  'saya',
  'berencana',
  'p2',
  'di',
  'minggu',
  'ini',
  'namun',
  'apabila',
  'dokumen',
  'skripsi',
  'saya',
  'masih',
  'belum',
  'selesai',
  'dapatkah',
  'saya',
  'melanjutkan',
  'skripsi',
  'ini',
  'di',
  'semester',
  'genap',
  'bapak',
  'dengan',
  'bapak',
  'dan',
  'ibu',
  '[dosen]',
  'sebagai',
  'pembimbing',
  'saya',
  'seperti',
  'saat',
  'ini',
  'mohon',
  'bantuannya',
  'bapak',
  'terima kasih',
  'sebelumnya',
  'mohon',
  'maaf',
  'mengganggu',
  'waktunya',
  'selamat',
  'siang'],
 ['walaikumsallam',
  'dikerjakan',
  'progres',
  'realnya',
  'saja',
  'nanti',
  'masalah',
  'administrasi',
  'sistem',
  'bisa',
  'diatur',
  'yang',
  'pe

In [126]:
def filtering(msg):
    with open('stopword_list_tala.txt', 'r') as tala:
        stoplist = tala.read()
    token_filtered = [[token for token in i if not token in stoplist] for i in msg]
    return token_filtered

token_filtered = filtering(token_normal)
token_filtered

[['selamat',
  'siang',
  '[dosen]',
  'mahasiswi',
  'bimbingan',
  'skripsi',
  'skripsi',
  'tahap',
  'p1',
  'tahap',
  'pembuatan',
  'prototype',
  'berencana',
  'p2',
  'minggu',
  'dokumen',
  'skripsi',
  'selesai',
  'dapatkah',
  'melanjutkan',
  'skripsi',
  'semester',
  'genap',
  '[dosen]',
  'pembimbing',
  'mohon',
  'bantuannya',
  'terima kasih',
  'mohon',
  'maaf',
  'mengganggu',
  'selamat',
  'siang'],
 ['walaikumsallam',
  'progres',
  'realnya',
  'administrasi',
  'sistem',
  'diatur',
  'kerjaan',
  'skripsinya',
  'jauh-jauh',
  'timeline',
  'target',
  'kemarin',
  'semester',
  'silahkan'],
 ['[dosen]', 'terima kasih', 'selesaikan']]

In [127]:
stemmer = StemmerFactory().create_stemmer()

def stemming(msg):
    token_stemmed = [[stemmer.stem(token) for token in i] for i in msg]
    return token_stemmed

token_stemmed = stemming(token_filtered)
token_stemmed

[['selamat',
  'siang',
  'dosen',
  'mahasiswi',
  'bimbing',
  'skripsi',
  'skripsi',
  'tahap',
  'p1',
  'tahap',
  'buat',
  'prototype',
  'rencana',
  'p2',
  'minggu',
  'dokumen',
  'skripsi',
  'selesai',
  'dapat',
  'lanjut',
  'skripsi',
  'semester',
  'genap',
  'dosen',
  'bimbing',
  'mohon',
  'bantu',
  'terima kasih',
  'mohon',
  'maaf',
  'ganggu',
  'selamat',
  'siang'],
 ['walaikumsallam',
  'progres',
  'real',
  'administrasi',
  'sistem',
  'atur',
  'kerja',
  'skripsi',
  'jauh',
  'timeline',
  'target',
  'kemarin',
  'semester',
  'silah'],
 ['dosen', 'terima kasih', 'selesai']]

In [128]:
setToken = [set(token) for token in token_stemmed]
setToken

[{'bantu',
  'bimbing',
  'buat',
  'dapat',
  'dokumen',
  'dosen',
  'ganggu',
  'genap',
  'lanjut',
  'maaf',
  'mahasiswi',
  'minggu',
  'mohon',
  'p1',
  'p2',
  'prototype',
  'rencana',
  'selamat',
  'selesai',
  'semester',
  'siang',
  'skripsi',
  'tahap',
  'terima kasih'},
 {'administrasi',
  'atur',
  'jauh',
  'kemarin',
  'kerja',
  'progres',
  'real',
  'semester',
  'silah',
  'sistem',
  'skripsi',
  'target',
  'timeline',
  'walaikumsallam'},
 {'dosen', 'selesai', 'terima kasih'}]

In [129]:
for i in setToken:
    print(len(i))

24
14
3
